### Задача: SQL Витрина  
1. У вас есть датасет транзакций (можно взять из прошлых уроков или сгенерировать).
2. Зарегистрируйте его как transactions.
3. Напишите один SQL запрос, который делает всё то, что мы делали в Уроке 4.2 (Окна):
    - Для каждой категории (category)
    - Найти топ-2 товара по цене (rank или row_number)
    - Посчитать разницу с предыдущим товаром (lag).

Подсказка: В Spark SQL оконные функции пишутся так: ROW_NUMBER() OVER (PARTITION BY category ORDER BY price DESC).

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("SQL_mart").getOrCreate()

data = [
    ("iPhone", "Mobile", 1000),
    ("Samsung", "Mobile", 800),
    ("Xiaomi", "Mobile", 800), # Цена совпадает с Samsung!
    ("Nokia", "Mobile", 300),
    ("MacBook", "Laptop", 2000),
    ("Dell", "Laptop", 1500),
    ("HP", "Laptop", 1200)
]

columns = ["product", "category", "price"]

transactions_df = spark.createDataFrame(data, columns)
transactions_df.show()

+-------+--------+-----+
|product|category|price|
+-------+--------+-----+
| iPhone|  Mobile| 1000|
|Samsung|  Mobile|  800|
| Xiaomi|  Mobile|  800|
|  Nokia|  Mobile|  300|
|MacBook|  Laptop| 2000|
|   Dell|  Laptop| 1500|
|     HP|  Laptop| 1200|
+-------+--------+-----+



In [2]:
transactions_df.createOrReplaceTempView("transactions")
trans_df = spark.sql("""
    WITH ranked_products AS (
    SELECT 
        category,
        product,
        price,
        ROW_NUMBER() OVER (PARTITION BY category ORDER BY price DESC) as rank_in_category,
        LAG(price) OVER (PARTITION BY category ORDER BY price DESC) as previous_price,
        price - LAG(price) OVER (PARTITION BY category ORDER BY price DESC) as price_diff_from_previous
    FROM transactions
)
SELECT 
    category,
    product,
    price,
    rank_in_category,
    previous_price,
    price_diff_from_previous
FROM ranked_products
WHERE rank_in_category <= 2
ORDER BY category, rank_in_category
""")

trans_df.show()

+--------+-------+-----+----------------+--------------+------------------------+
|category|product|price|rank_in_category|previous_price|price_diff_from_previous|
+--------+-------+-----+----------------+--------------+------------------------+
|  Laptop|MacBook| 2000|               1|          NULL|                    NULL|
|  Laptop|   Dell| 1500|               2|          2000|                    -500|
|  Mobile| iPhone| 1000|               1|          NULL|                    NULL|
|  Mobile|Samsung|  800|               2|          1000|                    -200|
+--------+-------+-----+----------------+--------------+------------------------+

